In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import csv 
from pathlib import Path
from os import listdir
import pickle

from labeling_utils import load_labels

import numpy as np
from sklearn.metrics import precision_recall_fscore_support


# tags=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Vehicle","Aircraft"]
tagSet=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Aircraft"]

from tabulate import tabulate


In [3]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [4]:
#LOAD MODEL predictions
splits_path= Path('/files/scratch/enis/data/nna/labeling/splits/')

# labelsbymodelpath=Path('/scratch/enis/data/nna/labeling/deep_labels/')

# dbfile = open(labelsbymodelpath / "classifications_all_2.pkl", 'rb')  
# modelresults = pickle.load(dbfile) 
# dbfile.close() 

In [5]:
# print(len(modelresults))
# aiter=iter(modelresults.keys())
# akey=next(aiter)
# print(akey)
# modelresults[akey]

In [5]:
# LOAD LABELS by human
labelsbyhumanpath=Path('/scratch/enis/data/nna/labeling/results/')
# filter by username
labelsbyhuman=[i for i in listdir(labelsbyhumanpath) if (".csv" in i) ]

humanresults={}
counter=0
for apath in labelsbyhuman:
    with open(labelsbyhumanpath / apath, newline='') as f:
        reader=csv.reader(f)
        for row in reader:
            counter+=1
            humanresults[row[0]]=row[1:]

print("unique files:",len(humanresults),"\ntotal files",counter)

unique files: 1300 
total files 1366


In [8]:
#Join vehicle and Aircraft
for file_name,tagshere in humanresults.items():
#     print(file_name,tagshere)
    
    tagshere=["Aircraft" if tag == "Vehicle" else tag for tag in tagSet]

In [11]:
# items={}
# from IPython.display import Audio,display,HTML
# from ipywidgets import Output

# items["mp3_output"]=Output()

# def play_html_modify(mp3file,items={}):
# #     out=items["mp3_output"]
# #     displayed=display(HTML("<audio controls  loop autoplay><source src={} type='audio/mpeg'></audio>".format(mp3file)))
#     displayed=display(HTML("<audio controls  loop><source src={} type='audio/mpeg'></audio>".format(mp3file)))


In [9]:
def vectorized_y_true(humanresults,tag_set):
    y_true={tag: [None]*len(humanresults) for tag in tag_set}
    for i,tags in enumerate(humanresults.values()):
        # we  only look for tags in tag_set
        for tag in tag_set:
            if tag in tags:
                y_true[tag][i] = 1
            else:
                y_true[tag][i] = 0 
    return y_true

In [10]:
y_true_dict=vectorized_y_true(humanresults,tag_set)


In [14]:
# for i,filename in enumerate(humanresults.keys()):
#     orig,audioop = modelresults[filename]


In [19]:
# class color:
#    PURPLE = '\033[95m'
#    CYAN = '\033[96m'
#    DARKCYAN = '\033[36m'
#    BLUE = '\033[94m'
#    GREEN = '\033[92m'
#    YELLOW = '\033[93m'
#    RED = '\033[91m'
#    BOLD = '\033[1m'
#    UNDERLINE = '\033[4m'
#    END = '\033[0m'
    
# short_names=['N. Neighbors',
#  'Linear SVM',
#  'RBF SVM',
#  'Gaussian P.',
#  'Decision T.',
#  'Random F.',
#  'NN',
#  'AdaBoost',
#  'Naive B.',
#  'QDA']

def print_result(results):
    headers= ["Tag",]+short_names
    table=[]
    for tag in (tag_set):
        table.append([tag,*results[tag]])

    print(tabulate(table, headers=headers))

def reverse_results(results,classifierNames=[]):
    results_clf2={}
    for tag,scores_per_Tag2 in results.items():
        scores_per_Tag=scores_per_Tag2[:]
        maximum=max(scores_per_Tag)
        scores_per_Tag=[color.BOLD + x + color.END if x==maximum else x for x in scores_per_Tag]

        for i,name in enumerate(classifierNames):
            score=scores_per_Tag[i]
            results_clf2.setdefault(name,[])
            results_clf2[name].append(score)      
    return results_clf2
        

def print_result2(results,classifierNames=[],tag_set=[]):
    results_clf=reverse_results(results,classifierNames)
    headers= ["Tag",]+list(tag_set)
    table=[]
#     sample_count=len(next(iter(y_true_dict.values())))
#     print("Total sample:",sample_count,"And threshold is",prob_threshold)
#     for tag in tag_set:
#         aresult=[]
    for i,name in enumerate(classifierNames):
#         positive_count=sum(y_true_dict[tag])
        table.append([classifierNames[i],*results_clf[name]])
    
    print(tabulate(table, headers=headers))


(1300, 10, 128)

In [22]:
def map_reduce(X,y_true_dict,func_type):
    X_copy=X[:]
    # X.shape --> (1300, 10, 128)
    y_true_dict_copy=copy.deepcopy(y_true_dict)
    if func_type=="Average":
        return np.mean(X_copy,axis=1),y_true_dict_copy
    elif func_type=="Concat":
        return np.reshape(X_copy,(-1,1280)),y_true_dict_copy
    elif func_type=="many2one":
        return np.reshape(X_copy,(-1,1280)),y_true_dict_copy
    else:
        raise Exception("ERROR with embed type")

def pick_embed(embed_type):
    # humanresults[proc_embeds[0].replace("_embed.npy",".mp3")]
    X=[]
    for i in humanresults:
        if embed_type=="Raw":
            file_name=i.replace(".mp3","_rawembed.npy")
        elif embed_type=="Normalized":
            file_name=i.replace(".mp3","_embed.npy")
        else:
            raise Exception("ERROR with embed type")
        an_x=np.load(split_path / file_name)
        X.append(an_x)
    return np.array(X)

In [23]:
# params
embed_types=["Raw","Normalized"] # "Raw" or "Normalized"
map_reduce_embeds= ["many2one","Concat","Average"] # Concat, Average



In [24]:
classifier2use = [
    KNeighborsClassifier(3),
#     SVC(probability=True,kernel="linear", C=0.025),
#     SVC(probability=True,gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=128),
    MLPClassifier(alpha=1, max_iter=1000),
    MLPClassifier(random_state=84,max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    ]

classifierNames2use = ["Nearest Neighbors", 
#                        "Linear SVM",
#                        "RBF SVM", 
#                    "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

all_results={}
for embed_type in embed_types:
    X=pick_embed(embed_type)
    for map_reduce_type in map_reduce_embeds:
        X_transformed,y_dict=map_reduce(X,y_true_dict,map_reduce_type)
        print("\n {} embeddings and merged by {} ".format(embed_type,map_reduce_type))
        many2one= True if map_reduce_type=="many2one" else False
        results,models=get_results(X_transformed,y_dict,classifier2use,classifierNames2use,['Insect',],logs=True,many2one=many2one)
        all_results[embed_type+"_"+map_reduce_type]=results
#         print_result2(results,classifierNames,['Aircraft',])



 Raw embeddings and merged by many2one 
Insect Nearest Neighbors 0.75
Insect Decision Tree 0.72
Insect Random Forest 0.75
Insect Neural Net 0.79
Insect AdaBoost 0.77
Insect Naive Bayes 0.74
Insect QDA 0.66

 Normalized embeddings and merged by many2one 
Insect Nearest Neighbors 0.75
Insect Decision Tree 0.71
Insect Random Forest 0.77
Insect Neural Net 0.78
Insect AdaBoost 0.75
Insect Naive Bayes 0.79
Insect QDA 0.75


### run best algorithm  for each tag and save


In [43]:
all_results={}
all_models={}
selected_tags={"Wind":("Raw","many2one",'Neural Net'),
               "Cable":("Raw","Concat","Gaussian Process"),
               "Songbird":("Raw","many2one","Neural Net"),
               'Running Water':("Raw","Concat","Neural Net"),
               'Water Bird':("Raw","many2one","RBF SVM"),
               'Insect':("Raw","Concat","Linear SVM"),
               'Rain':("Raw","Concat","Gaussian Process"),
               'Aircraft':("Raw","Average","AdaBoost")}

X=pick_embed("Raw")
previous_emded_type="Raw"
for tag,(embed_type,map_reduce_type,classifierName) in selected_tags.items():
    if previous_emded_type!=embed_type:
        X=pick_embed(embed_type)
        previous_emded_type=embed_type
    X_transformed,y_dict=map_reduce(X,y_true_dict,map_reduce_type)

    
    many2one= True if map_reduce_type=="many2one" else False
    classifier=classifiers[classifierNames.index(classifierName)]
    print("\n {} embeddings and merged by {} for {} - classifier {}".format(embed_type,map_reduce_type,tag,classifierName))

    results,models=get_results(X_transformed,y_dict,[classifier,],[classifierName,],[tag,],logs=False,many2one=many2one)
    all_results["_".join([embed_type,map_reduce_type,classifierName,tag])]=results
    all_models["_".join([embed_type,map_reduce_type,classifierName,tag])]=models
    print_result2(results,[classifierName,],[tag,])
    


 Raw embeddings and merged by many2one for Wind - classifier Neural Net
Tag           Wind
----------  ------
Neural Net    0.86

 Raw embeddings and merged by Concat for Cable - classifier Gaussian Process
Tag                 Cable
----------------  -------
Gaussian Process     0.85

 Raw embeddings and merged by many2one for Songbird - classifier Neural Net
Tag           Songbird
----------  ----------
Neural Net        0.85

 Raw embeddings and merged by Concat for Running Water - classifier Neural Net
Tag           Running Water
----------  ---------------
Neural Net             0.91

 Raw embeddings and merged by many2one for Water Bird - classifier RBF SVM
Tag        Water Bird
-------  ------------
RBF SVM          0.77

 Raw embeddings and merged by Concat for Insect - classifier Linear SVM
Tag           Insect
----------  --------
Linear SVM       0.8

 Raw embeddings and merged by Concat for Rain - classifier Gaussian Process
Tag                 Rain
----------------  ------

In [26]:
# all_results={}
# all_models={}
# selected_tags={
# #                 "Wind":("Raw","many2one",'Neural Net'),
# #                "Cable":("Raw","Concat","Gaussian Process"),
# #                "Songbird":("Raw","many2one","Neural Net"),
# #                'Running Water':("Raw","Concat","Neural Net"),
# #                'Water Bird':("Raw","many2one","Neural Net"),
# #                'Insect':("Raw","many2one","Linear SVM"),
# #                'Rain':("Raw","Concat","Neural Net"),
#                 'Aircraft':("Raw","Average","AdaBoost"),
#                 }

# X=pick_embed("Raw")
# previous_emded_type="Raw"
# for tag,(embed_type,map_reduce_type,classifierName) in selected_tags.items():
#     if previous_emded_type!=embed_type:
#         X=pick_embed(embed_type)
#         previous_emded_type=embed_type
#     X_transformed,y_dict=map_reduce(X,y_true_dict,map_reduce_type)


#     many2one= True if map_reduce_type=="many2one" else False
#     classifier=classifiers[classifierNames.index(classifierName)]
#     print("\n {} embeddings and merged by {} for {} - classifier {}".format(embed_type,map_reduce_type,tag,classifierName))

#     results,models=get_results(X_transformed,y_dict,[classifier,],[classifierName,],[tag,],logs=False,many2one=many2one)
#     all_results["_".join([embed_type,map_reduce_type,classifierName])]=results
#     all_models["_".join([embed_type,map_reduce_type,classifierName])]=models
#     print_result2(results,[classifierName,],[tag,])
    


 Raw embeddings and merged by Average for Aircraft - classifier AdaBoost
Tag         Aircraft
--------  ----------
AdaBoost        0.74


In [50]:
for sModel in SelectedModels:
    all_models[sModel]=list(all_models[sModel].values())[0]

    

In [53]:
[i.split("_") for i in all_models.keys()]

[['Raw', 'many2one', 'Neural Net', 'Wind'],
 ['Raw', 'Concat', 'Gaussian Process', 'Cable'],
 ['Raw', 'many2one', 'Neural Net', 'Songbird'],
 ['Raw', 'Concat', 'Neural Net', 'Running Water'],
 ['Raw', 'many2one', 'RBF SVM', 'Water Bird'],
 ['Raw', 'Concat', 'Linear SVM', 'Insect'],
 ['Raw', 'Concat', 'Gaussian Process', 'Rain'],
 ['Raw', 'Average', 'AdaBoost', 'Aircraft']]

In [62]:
from datetime import datetime
print(timestr)

2020-03-02--14-02


In [67]:
for sModel in list(all_models.keys()):
    model=all_models[sModel]
    embed_type,map_reduce_type,classifierName,tag=sModel.split("_")
    timestr=datetime.now().strftime("%Y-%m-%d--%H-%M")
    pickle_name="_".join([tag,classifierName,embed_type,map_reduce_type,timestr])
    pickle_name+=".joblib"
    pickle_name="/home/enis/projects/nna/src/assets/sklearnModels/"+pickle_name
    print(pickle_name)
    joblib.dump(model, pickle_name) 

/home/enis/projects/nna/src/assets/sklearnModels/Wind_Neural Net_Raw_many2one_2020-03-02--14-06.joblib
/home/enis/projects/nna/src/assets/sklearnModels/Cable_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib
/home/enis/projects/nna/src/assets/sklearnModels/Songbird_Neural Net_Raw_many2one_2020-03-02--14-06.joblib
/home/enis/projects/nna/src/assets/sklearnModels/Running Water_Neural Net_Raw_Concat_2020-03-02--14-06.joblib
/home/enis/projects/nna/src/assets/sklearnModels/Water Bird_RBF SVM_Raw_many2one_2020-03-02--14-06.joblib
/home/enis/projects/nna/src/assets/sklearnModels/Insect_Linear SVM_Raw_Concat_2020-03-02--14-06.joblib
/home/enis/projects/nna/src/assets/sklearnModels/Rain_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib
/home/enis/projects/nna/src/assets/sklearnModels/Aircraft_AdaBoost_Raw_Average_2020-03-02--14-06.joblib


In [267]:
filename="/scratch/enis/data/nna/real/ivvavik/SINP01/2019/SINP-01_20190516_033000_vgg/SINP-01_20190516_033000_rawembeddings000.npy"
modelPath="/home/enis/projects/nna/src/assets/sklearnModels/Wind_Neural Net_Raw_many2one_2020-03-02--14-06.joblib"
modelPath="/home/enis/projects/nna/src/assets/sklearnModels/Aircraft_AdaBoost_Raw_Average_2020-03-02--14-06.joblib"
modelPath="/home/enis/projects/nna/src/assets/sklearnModels/Insect_Linear SVM_Raw_Concat_2020-03-02--14-06.joblib"
model=joblib.load(modelPath)
rawembeddings=np.load(filename)
rawembeddings=rawembeddings[:4554,:]
output=modelPath.split("/")[-1].split("_")
tag,classifierName,embed_type,map_reduce_type,timestampStr=output
res=map_predict(rawembeddings,model,map_reduce_type)



IndentationError: unexpected indent (<ipython-input-267-c30258936285>, line 9)

In [186]:
rawembeddings[:-(rawembeddings.shape[0]%10)].shape

(4550, 128)